In [1]:
!pip install numpy 
import pandas as pd 
import numpy as np 
import os 
import matplotlib.pyplot as plt 
import seaborn as sns 
!pip install scikit-learn 
!pip uninstall numpy --yes 
!pip install numpy==1.26.0 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import roc_auc_score 
!pip install gensim --yes 
!pip install gensim 
import gensim 
from sklearn.metrics.pairwise import cosine_similarity 
import time 
from IPython.display import clear_output 

Found existing installation: numpy 1.26.0
Uninstalling numpy-1.26.0:
  Successfully uninstalled numpy-1.26.0
  Using cached numpy-1.26.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (53 kB)
Using cached numpy-1.26.0-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tableone 0.9.1 requires openpyxl>=3.1.2, which is not installed.

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --yes


In [2]:
filename = "/Users/jocelyn/Desktop/DXC-4 Technology /AI-Enhanced-Customer-Interaction-Assistant/data/Final_db.csv" 
df = pd.read_csv(filename, header=0) 

In [3]:
# check to see if there are products with multiple orders 
product_id_value_counts = df["product_id"].value_counts() 

# store the products with multiple orders in products_multiple_orders 
products_multiple_orders = product_id_value_counts[product_id_value_counts > 1] 

# create a new column in df that stores the number of times each item was purchased 
df["times_purchased"] = df["product_id"].map(product_id_value_counts) 

# stores all of the unique product categories into the array "product_categories" 
product_categories = df["product_cat_name_translated"].unique() 

# fill in rows in df["review_comment_title"] column that have na values with "" 
df["review_comment_title"] = df["review_comment_title"].fillna("") 

# fill in rows in df["review_comment_message"] column that have na values with "" 
df["review_comment_message"] = df["review_comment_message"].fillna("") 

# merge the product reviews and product review titles with a space in between both 
df["merged_review_messages_and_titles"] = df["review_comment_message"] + " " + df["review_comment_title"] 
df["merged_review_messages_and_titles"] = df["merged_review_messages_and_titles"].str.strip() 

# dictionary to store dataframes of the product reviews for all products in each category 
# products are separated by category 
product_reviews_by_category = {}; 

# fill in dictionary with keys (df category) and values (product_id and product reviews for all products in that category) 
for category in product_categories: 
    product_reviews_by_category["df_" + str(category)] = df[df["product_cat_name_translated"] == category][["product_id", "merged_review_messages_and_titles"]] 

# feature ~ product title and review 
X = df["merged_review_messages_and_titles"] 

# label ~ product category 
y = df["product_cat_name_translated"] 

# preprocess each (each row) product title and review 
X = X.apply(lambda row: gensim.utils.simple_preprocess(row)) 

# split data into a training set and a test set 
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.80, random_state=1111) 

word2vec_model = gensim.models.Word2Vec(X_train, vector_size=100, window=5, min_count=1) 

# replace every word in the dataset that's also found in word2vec_model with their word embedding 
# a set with all of the unique words in the word2vex_model 
words = set(word2vec_model.wv.index_to_key) 

# X_train word embeddings 
# array for all of the word embeddings for X_train 
X_train_word_embeddings = []; 
for row in X_train: 
    row_embedding = [] 
    for word in words: 
        if word in row: 
            row_embedding.append(word2vec_model.wv[word]) 
    X_train_word_embeddings.append(np.array(row_embedding)) 

# X_test word embeddings 
# array for all of the word embeddings for X_test 
X_test_word_embeddings = []; 
for row in X_test: 
    row_embedding = []; 
    for word in words: 
        if word in row: 
            row_embedding.append(word2vec_model.wv[word]) 
    X_test_word_embeddings.append(np.array(row_embedding)) 

X_train_feature_vector = []
for w in X_train_word_embeddings:
    if w.size:
        X_train_feature_vector.append(w.mean(axis=0))
    else:
        X_train_feature_vector.append(np.zeros(100, dtype=float))
        
X_test_feature_vector = []
for w in X_test_word_embeddings:
    if w.size:
        X_test_feature_vector.append(w.mean(axis=0))
    else:
        X_test_feature_vector.append(np.zeros(100, dtype=float))

print(X_train_feature_vector[1:11]) 

# logistic regression 
model = LogisticRegression(max_iter=500)
model.fit(X_train_feature_vector, y_train)

probability_predictions = model.predict_proba(X_test_feature_vector) 

# predict most likely product category 
class_label_predictions = model.predict(X_test_feature_vector) 

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([-0.10439922,  0.13158454, -0.2001088 ,  0.01089589, -0.06955387,
       -0.4117542 ,  0.01581176,  0.91423404, -0.71401894, -0.85532063,
       -0.09743784, -0.10605977,  0.20183387,  0.02999123, -0.00934011,
       -0.08430766,  0.38189006, -0.22335796, -0.24631536, -0.61558723,
        0.1583482 , -0.5293387 ,  0.608945  , -0.03829361, -0.20074442,
        0.37492   , -0.49715137, -0.33835596, -0.263556  ,  0.19799344,
        0.27581564,  0.14259148, -0.05737156, -0.35996908, -0.20203114,
        0.18417025, -0.20313096, -0.17077069, -0.00

In [6]:
print("Hello! Welcome to the personalized product recommendation chatbot.") 

# create a list containing the vectors for the tokens/words found in the user input that are also found in the dataset of available products 
token_vectors = [] 

while len(token_vectors) == 0: 
    another = ""
    user_input = input("\nWhat product are you looking for?") 

    print("Generating response ...") 

    user_input_tokens = list(gensim.utils.simple_preprocess(user_input)) 
    token_vectors = [] 
    
    # loop through all tokens from the tokenized version of the user input 
    for token in user_input_tokens: 
        # if the token is also found in the dataset 
        if token in word2vec_model.wv.key_to_index: 
            # add its word vector to the "token_vectors" list 
            token_vectors.append(word2vec_model.wv[token]) 
    
    # if there are vectors 
    if len(token_vectors) > 0: 
        # calculate the mean for each token_vector and combine into an array 
        overall_token_vector = np.mean(token_vectors, axis=0) 
        # reshape into a two dimensional array 
        overall_token_vector = overall_token_vector.reshape(1, -1) 
        word2vec_prediction = model.predict(overall_token_vector) 
        print(type(word2vec_prediction)) 
    

Hello! Welcome to the personalized product recommendation chatbot.



What product are you looking for? book


Generating response ...
<class 'numpy.ndarray'>


In [4]:
print("Hello! Welcome to the personalized product recommendation chatbot.") 

# create a list containing the vectors for the tokens/words found in the user input that are also found in the dataset of available products 
token_vectors = [] 

while len(token_vectors) == 0: 
    another = ""
    user_input = input("\nWhat product are you looking for?") 

    print("Generating response ...") 

    user_input_tokens = list(gensim.utils.simple_preprocess(user_input)) 
    token_vectors = [] 
    
    # loop through all tokens from the tokenized version of the user input 
    for token in user_input_tokens: 
        # if the token is also found in the dataset 
        if token in word2vec_model.wv.key_to_index: 
            # add its word vector to the "token_vectors" list 
            token_vectors.append(word2vec_model.wv[token]) 
    
    # if there are vectors 
    if len(token_vectors) > 0: 
        # calculate the mean for each token_vector and combine into an array 
        overall_token_vector = np.mean(token_vectors, axis=0) 
        # reshape into a two dimensional array 
        overall_token_vector = overall_token_vector.reshape(1, -1) 
        word2vec_prediction = model.predict(overall_token_vector) 
        print(word2vec_prediction) 
    
    else: 
        user_input = input("\nWe can't find a related product. Please input more detail.") 

    word2vec_prediction_str = "df_" + str(word2vec_prediction[0]) 
    
    #vectorizer = TfidfVectorizer(ngram_range=(2, 3)) 
    
    product_reviews_by_category[word2vec_prediction_str] 
    
    # use bigrams (for cases of negation like "not good") and trigrams (for cases with numbers like "bought 3 pairs") 
    vectorizer = TfidfVectorizer(ngram_range=(2, 3)) 
    
    # fit vectorizer to the product review titles and messages 
    vectorizer.fit(product_reviews_by_category[word2vec_prediction_str]["merged_review_messages_and_titles"]) 
    
    # result_matrix contains each document's TF_IDF scores 
    resulting_matrix = vectorizer.transform(product_reviews_by_category[word2vec_prediction_str]["merged_review_messages_and_titles"]) 
        
    product_id = product_reviews_by_category[word2vec_prediction_str]["product_id"].tolist() 
        
    resulting_matrix = pd.DataFrame(resulting_matrix.toarray(), index=product_id, columns=vectorizer.get_feature_names_out()) 
    
    # determine product in category that's most similar to user input 
    user_input_TF_IDF_vector = vectorizer.transform([user_input]) 
    # similarity of user input word vector and each of the rows' word vectors 
    similarity = cosine_similarity(user_input_TF_IDF_vector, resulting_matrix) 
    greatest_similarity_indices = np.argsort(similarity[0][::-1]) 
    greatest_similarity_product_ids = resulting_matrix.index[greatest_similarity_indices] 
    greatest_similarity_product_ids_list = greatest_similarity_product_ids.unique().tolist() 
    
    # rows with that product id 
    products_df = df[df["product_id"] == greatest_similarity_product_ids_list[0]] 
    
    clear_output(wait=True) 
    
    print("\nHow about this product?") 
    print("Product: ", greatest_similarity_product_ids_list[0]) 
    print("Price: ", products_df["price"].iloc[0]) 
    print("Average Product Rating: ", products_df["review_score"].mean()) 
    
    more_detail_user_input = input("\nDo you want more detail about this product? (Y/N)") 
    
    more_detail_user_input = more_detail_user_input.upper().strip() 
    
    more_recommendations = "N" 
    more_detail = "" 
    moree = False 
    user_input_product_cat = "" 
    user_input = "" 
    
    if more_detail_user_input == "Y": 
        print("\nOk. Here is more detail about the product.") 
        print("Total Orders: ", len(products_df)) 
        print("Product Weight (g): ", products_df["product_weight_g"].iloc[0]) 
        print("Product Length (cm): ", products_df["product_length_cm"].iloc[0]) 
        print("Product Height (cm): ", products_df["product_height_cm"].iloc[0]) 
        print("Product Width (cm): ", products_df["product_width_cm"].iloc[0]) 
        
        good = input("\nIs this a good recommendation? (Y/N)") 
        good = good.upper().strip() 
    
        if good == "Y": 
            another = input("\nWe're glad! Would you like like help finding another product? (Y/N)") 
            another = another.upper().strip() 
            
            if another == "Y": 
                token_vectors = []; 
            else: 
                print("\nHave a great day! :)") 
                break 
        
        else: 
            print("\nWe're sorry.") 
            token_vectors = []; 
    
    else: 
        more_recommendations = input("\nWould you like more similar product recommendations? (Y/N)") 
        more_recommendations = more_recommendations.upper().strip() 
    
    if more_recommendations == "Y": 
        print("\nOk. Here are some more product recommendations.") 
        if (len(greatest_similarity_product_ids_list) == 3) | (len(greatest_similarity_product_ids_list) > 3): 
            products_df = df[df["product_id"] == greatest_similarity_product_ids_list[1]] 
        
            print("Product: ", greatest_similarity_product_ids_list[0]) 
            print("Price: ", products_df["price"].iloc[0]) 
            print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 
    
            products_df = df[df["product_id"] == greatest_similarity_product_ids_list[1]] 
            print("Product: ", greatest_similarity_product_ids_list[0]) 
            print("Price: ", products_df["price"].iloc[0]) 
            print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 
    
            products_df = df[df["product_id"] == greatest_similarity_product_ids_list[2]] 
            print("Product: ", greatest_similarity_product_ids_list[0]) 
            print("Price: ", products_df["price"].iloc[0]) 
            print("Average Product Rating: ", products_df["review_score"].mean()) 
    
            good = input("\nWere any of these recommendations helpful? (Y/N)") 
            good = good.upper().strip() 
    
            if good == "Y": 
                another = input("\nWe're glad! Would you like like help finding another product? (Y/N)") 
                another = another.upper().strip() 
                
                if another == "Y": 
                    token_vectors = []; 
                else: 
                    print("\nHave a great day! :)") 
                    break 
            
            else: 
                token_vectors = []; 
                print("\nWe're sorry.") 
    
        elif len(greatest_similarity_product_ids_list) == 2: 
            products_df = df[df["product_id"] == greatest_similarity_product_ids_list[1]] 
        
            print("Product: ", greatest_similarity_product_ids_list[0]) 
            print("Price: ", products_df["price"].iloc[0]) 
            print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 
    
            products_df = df[df["product_id"] == greatest_similarity_product_ids_list[1]] 
            print("Product: ", greatest_similarity_product_ids_list[0]) 
            print("Price: ", products_df["price"].iloc[0]) 
            print("Average Product Rating: ", products_df["review_score"].mean()) 
    
            good = input("\nWere any of these recommendations helpful? (Y/N)") 
            good = good.upper().strip() 
    
            if good == "Y": 
                another = input("\nWe're glad! Would you like like help finding another product? (Y/N)") 
                another = another.upper().strip() 
                
                if another == "Y": 
                    token_vectors = []; 
                else: 
                    print("\nHave a great day! :)") 
                    break 
            
            else: 
                token_vectors = []; 
                print("\nWe're sorry.") 
    
        else: 
            moree = True 
            
            more_detail = input("\nThere are no more similar products. What category is your product in? Enter number.") 
    
            # a dictionary with a number associated with each product category 
            product_categories_dict = {} 
        
            # array with unique product categories 
            product_categories = df["product_cat_name_translated"].unique() 
        
            for i in range(len(product_categories)): 
                product_categories_dict[i+1] = product_categories[i] 
        
            # ask user for product category 
            print("\nWhich category is the product in? Enter in number.") 
            for keys, values in product_categories_dict.items(): 
                print(f"{keys}: {values}") 
            
            user_input_2 = input("") 
            
            user_input_2 = int(user_input_2.strip()) 
            
            user_input_product_cat = product_categories_dict.get(user_input_2) 

    else: 
        if (more_detail_user_input != "Y") & (another != "Y"): 
            print("\nHave a great day! :)") 
            break 
    
    if moree == True: 
        user_product_category = df[df["product_cat_name_translated"] == user_input_product_cat] 
        ranked_products = user_product_category.sort_values(by="review_score", ascending=False) 
        
        print("\nHere are the three highest ranked products in that category.") 
        print(ranked_products.head(3)["product_id"]) 

Hello! Welcome to the personalized product recommendation chatbot.



What product are you looking for? health


Generating response ...



We can't find a related product. Please input more detail. book


NameError: name 'word2vec_prediction' is not defined

In [7]:
print("Hello! Welcome to the personalized product recommendation chatbot.") 

# create a list containing the vectors for the tokens/words found in the user input that are also found in the dataset of available products 
token_vectors = [] 

while len(token_vectors) == 0: 
    another = ""
    user_input = input("\nWhat product are you looking for?") 

    print("Generating response ...") 

    user_input_tokens = list(gensim.utils.simple_preprocess(user_input)) 
    token_vectors = [] 
    
    # loop through all tokens from the tokenized version of the user input 
    for token in user_input_tokens: 
        # if the token is also found in the dataset 
        if token in word2vec_model.wv.key_to_index: 
            # add its word vector to the "token_vectors" list 
            token_vectors.append(word2vec_model.wv[token]) 
    
    # if there are vectors 
    if len(token_vectors) > 0: 
        # calculate the mean for each token_vector and combine into an array 
        overall_token_vector = np.mean(token_vectors, axis=0) 
        # reshape into a two dimensional array 
        overall_token_vector = overall_token_vector.reshape(1, -1) 
        word2vec_prediction = model.predict(overall_token_vector) 
    
    else: 
        while len(token_vectors) == 0: 
            user_input = input("\nWe can't find a related product. Please input more detail.") 
            user_input_tokens = list(gensim.utils.simple_preprocess(user_input)) 
            token_vectors = [] 
    
            # loop through all tokens from the tokenized version of the user input 
            for token in user_input_tokens: 
                # if the token is also found in the dataset 
                if token in word2vec_model.wv.key_to_index: 
                    # add its word vector to the "token_vectors" list 
                    token_vectors.append(word2vec_model.wv[token]) 
    
            # if there are vectors 
            if len(token_vectors) > 0: 
                # calculate the mean for each token_vector and combine into an array 
                overall_token_vector = np.mean(token_vectors, axis=0) 
                # reshape into a two dimensional array 
                overall_token_vector = overall_token_vector.reshape(1, -1) 
                word2vec_prediction = model.predict(overall_token_vector) 

word2vec_prediction_str = "df_" + str(word2vec_prediction[0]) 
    
#vectorizer = TfidfVectorizer(ngram_range=(2, 3)) 
    
product_reviews_by_category[word2vec_prediction_str] 
    
# use bigrams (for cases of negation like "not good") and trigrams (for cases with numbers like "bought 3 pairs") 
vectorizer = TfidfVectorizer(ngram_range=(2, 3)) 
    
# fit vectorizer to the product review titles and messages 
vectorizer.fit(product_reviews_by_category[word2vec_prediction_str]["merged_review_messages_and_titles"]) 
    
# result_matrix contains each document's TF_IDF scores 
resulting_matrix = vectorizer.transform(product_reviews_by_category[word2vec_prediction_str]["merged_review_messages_and_titles"]) 
        
product_id = product_reviews_by_category[word2vec_prediction_str]["product_id"].tolist() 
        
resulting_matrix = pd.DataFrame(resulting_matrix.toarray(), index=product_id, columns=vectorizer.get_feature_names_out()) 
    
# determine product in category that's most similar to user input 
user_input_TF_IDF_vector = vectorizer.transform([user_input]) 
# similarity of user input word vector and each of the rows' word vectors 
similarity = cosine_similarity(user_input_TF_IDF_vector, resulting_matrix) 
greatest_similarity_indices = np.argsort(similarity[0][::-1]) 
greatest_similarity_product_ids = resulting_matrix.index[greatest_similarity_indices] 
greatest_similarity_product_ids_list = greatest_similarity_product_ids.unique().tolist() 
    
# rows with that product id 
products_df = df[df["product_id"] == greatest_similarity_product_ids_list[0]] 
    
clear_output(wait=True) 
    
print("\nHow about this product?") 
print("Product: ", greatest_similarity_product_ids_list[0]) 
print("Price: ", products_df["price"].iloc[0]) 
print("Average Product Rating: ", products_df["review_score"].mean()) 
    
more_detail_user_input = input("\nDo you want more detail about this product? (Y/N)") 
    
more_detail_user_input = more_detail_user_input.upper().strip() 
    
more_recommendations = "N" 
more_detail = "" 
moree = False 
user_input_product_cat = "" 
user_input = "" 
    
if more_detail_user_input == "Y": 
    print("\nOk. Here is more detail about the product.") 
    print("Total Orders: ", len(products_df)) 
    print("Product Weight (g): ", products_df["product_weight_g"].iloc[0]) 
    print("Product Length (cm): ", products_df["product_length_cm"].iloc[0]) 
    print("Product Height (cm): ", products_df["product_height_cm"].iloc[0]) 
    print("Product Width (cm): ", products_df["product_width_cm"].iloc[0]) 
        
    good = input("\nIs this a good recommendation? (Y/N)") 
    good = good.upper().strip() 
    
    if good == "Y": 
        another = input("\nWe're glad! Would you like like help finding another product? (Y/N)") 
        another = another.upper().strip() 
            
        if another == "Y": 
            token_vectors = []; 
        else: 
            print("\nHave a great day! :)") 
        
    else: 
        print("\nWe're sorry.") 
        token_vectors = []; 

else: 
    more_recommendations = input("\nWould you like more similar product recommendations? (Y/N)") 
    more_recommendations = more_recommendations.upper().strip() 
    
if more_recommendations == "Y": 
    print("\nOk. Here are some more product recommendations.") 
    if (len(greatest_similarity_product_ids_list) == 3) | (len(greatest_similarity_product_ids_list) > 3): 
        products_df = df[df["product_id"] == greatest_similarity_product_ids_list[1]] 
        
        print("Product: ", greatest_similarity_product_ids_list[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 
    
        products_df = df[df["product_id"] == greatest_similarity_product_ids_list[1]] 
        print("Product: ", greatest_similarity_product_ids_list[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 
    
        products_df = df[df["product_id"] == greatest_similarity_product_ids_list[2]] 
        print("Product: ", greatest_similarity_product_ids_list[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean()) 
    
        good = input("\nWere any of these recommendations helpful? (Y/N)") 
        good = good.upper().strip() 
    
        if good == "Y": 
            another = input("\nWe're glad! Would you like like help finding another product? (Y/N)") 
            another = another.upper().strip() 
                
            if another == "Y": 
                token_vectors = []; 
            else: 
                print("\nHave a great day! :)") 
            
        else: 
            token_vectors = []; 
            print("\nWe're sorry.") 
    
    elif len(greatest_similarity_product_ids_list) == 2: 
        products_df = df[df["product_id"] == greatest_similarity_product_ids_list[1]] 
        
        print("Product: ", greatest_similarity_product_ids_list[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 
    
        products_df = df[df["product_id"] == greatest_similarity_product_ids_list[1]] 
        print("Product: ", greatest_similarity_product_ids_list[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean()) 
    
        good = input("\nWere any of these recommendations helpful? (Y/N)") 
        good = good.upper().strip() 
    
        if good == "Y": 
            another = input("\nWe're glad! Would you like like help finding another product? (Y/N)") 
            another = another.upper().strip() 
                
            if another == "Y": 
                token_vectors = []; 
            else: 
                print("\nHave a great day! :)") 
            
        else: 
            token_vectors = []; 
            print("\nWe're sorry.") 
    
    else: 
        moree = True 
            
        more_detail = input("\nThere are no more similar products. What category is your product in? Enter number.") 
    
        # a dictionary with a number associated with each product category 
        product_categories_dict = {} 
        
        # array with unique product categories 
        product_categories = df["product_cat_name_translated"].unique() 
        
        for i in range(len(product_categories)): 
            product_categories_dict[i+1] = product_categories[i] 
        
        # ask user for product category 
        print("\nWhich category is the product in? Enter in number.") 
        for keys, values in product_categories_dict.items(): 
            print(f"{keys}: {values}") 
            
        user_input_2 = input("") 
            
        user_input_2 = int(user_input_2.strip()) 
            
        user_input_product_cat = product_categories_dict.get(user_input_2) 

else: 
    if (more_detail_user_input != "Y") & (another != "Y"): 
        print("\nHave a great day! :)") 
    
if moree == True: 
    user_product_category = df[df["product_cat_name_translated"] == user_input_product_cat] 
    ranked_products = user_product_category.sort_values(by="review_score", ascending=False) 
        
    print("\nHere are the three highest ranked products in that category.") 
    print(ranked_products.head(3)["product_id"]) 


How about this product?
Product:  10adb53d8faa890ca7c2f0cbcb68d777
Price:  19.9
Average Product Rating:  3.85


KeyboardInterrupt: Interrupted by user

In [6]:
print("Hello! Welcome to the personalized product recommendation chatbot.") 

# create a list containing the vectors for the tokens/words found in the user input that are also found in the dataset of available products 
token_vectors = [] 

while len(token_vectors) == 0: 
    another = ""
    user_input = input("\nWhat product are you looking for?") 

    print("Generating response ...") 

    user_input_tokens = list(gensim.utils.simple_preprocess(user_input)) 
    token_vectors = [] 
    
    # loop through all tokens from the tokenized version of the user input 
    for token in user_input_tokens: 
        # if the token is also found in the dataset 
        if token in word2vec_model.wv.key_to_index: 
            # add its word vector to the "token_vectors" list 
            token_vectors.append(word2vec_model.wv[token]) 
    
    # if there are vectors 
    if len(token_vectors) > 0: 
        # calculate the mean for each token_vector and combine into an array 
        overall_token_vector = np.mean(token_vectors, axis=0) 
        # reshape into a two dimensional array 
        overall_token_vector = overall_token_vector.reshape(1, -1) 
        word2vec_prediction = model.predict(overall_token_vector) 
    
    else: 
        while len(token_vectors) == 0: 
            user_input = input("\nWe can't find a related product. Please input more detail.") 
            user_input_tokens = list(gensim.utils.simple_preprocess(user_input)) 
            token_vectors = [] 
    
            # loop through all tokens from the tokenized version of the user input 
            for token in user_input_tokens: 
                # if the token is also found in the dataset 
                if token in word2vec_model.wv.key_to_index: 
                    # add its word vector to the "token_vectors" list 
                    token_vectors.append(word2vec_model.wv[token]) 
    
            # if there are vectors 
            if len(token_vectors) > 0: 
                # calculate the mean for each token_vector and combine into an array 
                overall_token_vector = np.mean(token_vectors, axis=0) 
                # reshape into a two dimensional array 
                overall_token_vector = overall_token_vector.reshape(1, -1) 
                word2vec_prediction = model.predict(overall_token_vector) 

word2vec_prediction_str = "df_" + str(word2vec_prediction[0]) 
    
#vectorizer = TfidfVectorizer(ngram_range=(2, 3)) 
    
product_reviews_by_category[word2vec_prediction_str] 
    
# use bigrams (for cases of negation like "not good") and trigrams (for cases with numbers like "bought 3 pairs") 
vectorizer = TfidfVectorizer(ngram_range=(2, 3)) 
    
# fit vectorizer to the product review titles and messages 
vectorizer.fit(product_reviews_by_category[word2vec_prediction_str]["merged_review_messages_and_titles"]) 
    
# result_matrix contains each document's TF_IDF scores 
resulting_matrix = vectorizer.transform(product_reviews_by_category[word2vec_prediction_str]["merged_review_messages_and_titles"]) 
        
product_id = product_reviews_by_category[word2vec_prediction_str]["product_id"].tolist() 
        
resulting_matrix = pd.DataFrame(resulting_matrix.toarray(), index=product_id, columns=vectorizer.get_feature_names_out()) 
    
# determine product in category that's most similar to user input 
user_input_TF_IDF_vector = vectorizer.transform([user_input]) 
# similarity of user input word vector and each of the rows' word vectors 
similarity = cosine_similarity(user_input_TF_IDF_vector, resulting_matrix) 
greatest_similarity_indices = np.argsort(similarity[0][::-1]) 
greatest_similarity_product_ids = resulting_matrix.index[greatest_similarity_indices] 
greatest_similarity_product_ids_list = greatest_similarity_product_ids.unique().tolist() 
    
# rows with that product id 
products_df = df[df["product_id"] == greatest_similarity_product_ids_list[0]] 
    
clear_output(wait=True) 
    
print("\nHow about this product?") 
print("Product: ", greatest_similarity_product_ids_list[0]) 
print("Price: ", products_df["price"].iloc[0]) 
print("Average Product Rating: ", products_df["review_score"].mean()) 
    
more_detail_user_input = input("\nDo you want more detail about this product? (Y/N)") 
    
more_detail_user_input = more_detail_user_input.upper().strip() 
    
more_recommendations = "N" 
more_detail = "" 
moree = False 
user_input_product_cat = "" 
user_input = "" 
    
if more_detail_user_input == "Y": 
    print("\nOk. Here is more detail about the product.") 
    print("Total Orders: ", len(products_df)) 
    print("Product Weight (g): ", products_df["product_weight_g"].iloc[0]) 
    print("Product Length (cm): ", products_df["product_length_cm"].iloc[0]) 
    print("Product Height (cm): ", products_df["product_height_cm"].iloc[0]) 
    print("Product Width (cm): ", products_df["product_width_cm"].iloc[0]) 
        
    good = input("\nIs this a good recommendation? (Y/N)") 
    good = good.upper().strip() 
    
    if good == "Y": 
        another = input("\nWe're glad! Would you like like help finding another product? (Y/N)") 
        another = another.upper().strip() 
            
        if another == "Y": 
            token_vectors = []; 
        else: 
            print("\nHave a great day! :)") 
        
    else: 
        print("\nWe're sorry.") 
        token_vectors = []; 

else: 
    more_recommendations = input("\nWould you like more similar product recommendations? (Y/N)") 
    more_recommendations = more_recommendations.upper().strip() 
    
if more_recommendations == "Y": 
    print("\nOk. Here are some more product recommendations.") 
    if (len(greatest_similarity_product_ids_list) == 3) | (len(greatest_similarity_product_ids_list) > 3): 
        products_df = df[df["product_id"] == greatest_similarity_product_ids_list[1]] 
        
        print("Product: ", greatest_similarity_product_ids_list[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 
    
        products_df = df[df["product_id"] == greatest_similarity_product_ids_list[1]] 
        print("Product: ", greatest_similarity_product_ids_list[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 
    
        products_df = df[df["product_id"] == greatest_similarity_product_ids_list[2]] 
        print("Product: ", greatest_similarity_product_ids_list[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean()) 
    
        good = input("\nWere any of these recommendations helpful? (Y/N)") 
        good = good.upper().strip() 
    
        if good == "Y": 
            another = input("\nWe're glad! Would you like like help finding another product? (Y/N)") 
            another = another.upper().strip() 
                
            if another == "Y": 
                token_vectors = []; 
            else: 
                print("\nHave a great day! :)") 
            
        else: 
            token_vectors = []; 
            print("\nWe're sorry.") 
    
    elif len(greatest_similarity_product_ids_list) == 2: 
        products_df = df[df["product_id"] == greatest_similarity_product_ids_list[1]] 
        
        print("Product: ", greatest_similarity_product_ids_list[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 
    
        products_df = df[df["product_id"] == greatest_similarity_product_ids_list[1]] 
        print("Product: ", greatest_similarity_product_ids_list[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean()) 
    
        good = input("\nWere any of these recommendations helpful? (Y/N)") 
        good = good.upper().strip() 
    
        if good == "Y": 
            another = input("\nWe're glad! Would you like like help finding another product? (Y/N)") 
            another = another.upper().strip() 
                
            if another == "Y": 
                token_vectors = []; 
            else: 
                print("\nHave a great day! :)") 
            
        else: 
            token_vectors = []; 
            print("\nWe're sorry.") 
    
    else: 
        moree = True 
            
        more_detail = input("\nThere are no more similar products. What category is your product in? Enter number.") 
    
        # a dictionary with a number associated with each product category 
        product_categories_dict = {} 
        
        # array with unique product categories 
        product_categories = df["product_cat_name_translated"].unique() 
        
        for i in range(len(product_categories)): 
            product_categories_dict[i+1] = product_categories[i] 
        
        # ask user for product category 
        print("\nWhich category is the product in? Enter in number.") 
        for keys, values in product_categories_dict.items(): 
            print(f"{keys}: {values}") 
            
        user_input_2 = input("") 
            
        user_input_2 = int(user_input_2.strip()) 
            
        user_input_product_cat = product_categories_dict.get(user_input_2) 

else: 
    if (more_detail_user_input != "Y") & (another != "Y"): 
        print("\nHave a great day! :)") 
    
if moree == True: 
    user_product_category = df[df["product_cat_name_translated"] == user_input_product_cat] 
    ranked_products = user_product_category.sort_values(by="review_score", ascending=False) 
        
    print("\nHere are the three highest ranked products in that category.") 
    print(ranked_products.head(3)["product_id"]) 


How about this product?
Product:  10adb53d8faa890ca7c2f0cbcb68d777
Price:  19.9
Average Product Rating:  3.85


KeyboardInterrupt: Interrupted by user

In [ ]:
print("Hello! Welcome to the personalized product recommendation chatbot.") 

# create a list containing the vectors for the tokens/words found in the user input that are also found in the dataset of available products 
token_vectors = [] 

while len(token_vectors) == 0: 
    another = ""
    user_input = input("\nWhat product are you looking for?") 

    print("Generating response ...") 

    user_input_tokens = list(gensim.utils.simple_preprocess(user_input)) 
    token_vectors = [] 
    
    # loop through all tokens from the tokenized version of the user input 
    for token in user_input_tokens: 
        # if the token is also found in the dataset 
        if token in word2vec_model.wv.key_to_index: 
            # add its word vector to the "token_vectors" list 
            token_vectors.append(word2vec_model.wv[token]) 
    
    # if there are vectors 
    if len(token_vectors) > 0: 
        # calculate the mean for each token_vector and combine into an array 
        overall_token_vector = np.mean(token_vectors, axis=0) 
        # reshape into a two dimensional array 
        overall_token_vector = overall_token_vector.reshape(1, -1) 
        word2vec_prediction = model.predict(overall_token_vector) 

    else: 
        while len(token_vectors) == 0: 
            user_input = input("\nWe can't find a related product. Please input more detail.") 
            user_input_tokens = list(gensim.utils.simple_preprocess(user_input)) 
            token_vectors = [] 
    
            # loop through all tokens from the tokenized version of the user input 
            for token in user_input_tokens: 
                # if the token is also found in the dataset 
                if token in word2vec_model.wv.key_to_index: 
                    # add its word vector to the "token_vectors" list 
                    token_vectors.append(word2vec_model.wv[token]) 
    
            # if there are vectors 
            if len(token_vectors) > 0: 
                # calculate the mean for each token_vector and combine into an array 
                overall_token_vector = np.mean(token_vectors, axis=0) 
                # reshape into a two dimensional array 
                overall_token_vector = overall_token_vector.reshape(1, -1) 
                word2vec_prediction = model.predict(overall_token_vector) 

    word2vec_prediction_str = "df_" + str(word2vec_prediction[0]) 
    
    #vectorizer = TfidfVectorizer(ngram_range=(2, 3)) 
    
    product_reviews_by_category[word2vec_prediction_str] 
    
    # use bigrams (for cases of negation like "not good") and trigrams (for cases with numbers like "bought 3 pairs") 
    vectorizer = TfidfVectorizer(ngram_range=(2, 3)) 
    
    # fit vectorizer to the product review titles and messages 
    vectorizer.fit(product_reviews_by_category[word2vec_prediction_str]["merged_review_messages_and_titles"]) 
    
    # result_matrix contains each document's TF_IDF scores 
    resulting_matrix = vectorizer.transform(product_reviews_by_category[word2vec_prediction_str]["merged_review_messages_and_titles"]) 
        
    product_id = product_reviews_by_category[word2vec_prediction_str]["product_id"].tolist() 
        
    resulting_matrix = pd.DataFrame(resulting_matrix.toarray(), index=product_id, columns=vectorizer.get_feature_names_out()) 
    
    # determine product in category that's most similar to user input 
    user_input_TF_IDF_vector = vectorizer.transform([user_input]) 
    # similarity of user input word vector and each of the rows' word vectors 
    similarity = cosine_similarity(user_input_TF_IDF_vector, resulting_matrix) 
    greatest_similarity_indices = np.argsort(similarity[0][::-1]) 
    greatest_similarity_product_ids = resulting_matrix.index[greatest_similarity_indices] 
    greatest_similarity_product_ids_list = greatest_similarity_product_ids.unique().tolist() 
    
    # rows with that product id 
    products_df = df[df["product_id"] == greatest_similarity_product_ids_list[0]] 
    
    clear_output(wait=True) 
    
    print("\nHow about this product?") 
    print("Product: ", greatest_similarity_product_ids_list[0]) 
    print("Price: ", products_df["price"].iloc[0]) 
    print("Average Product Rating: ", products_df["review_score"].mean()) 
    
    more_detail_user_input = input("\nDo you want more detail about this product? (Y/N)") 
    
    more_detail_user_input = more_detail_user_input.upper().strip() 
    
    more_recommendations = "N" 
    more_detail = "" 
    moree = False 
    user_input_product_cat = "" 
    user_input = "" 
    
    if more_detail_user_input == "Y": 
        print("\nOk. Here is more detail about the product.") 
        print("Total Orders: ", len(products_df)) 
        print("Product Weight (g): ", products_df["product_weight_g"].iloc[0]) 
        print("Product Length (cm): ", products_df["product_length_cm"].iloc[0]) 
        print("Product Height (cm): ", products_df["product_height_cm"].iloc[0]) 
        print("Product Width (cm): ", products_df["product_width_cm"].iloc[0]) 
        
        good = input("\nIs this a good recommendation? (Y/N)") 
        good = good.upper().strip() 
    
        if good == "Y": 
            another = input("\nWe're glad! Would you like like help finding another product? (Y/N)") 
            another = another.upper().strip() 
            
            if another == "Y": 
                token_vectors = []; 
            else: 
                print("\nHave a great day! :)") 
                break 
        
        else: 
            print("\nWe're sorry.") 
            token_vectors = []; 
    
    else: 
        more_recommendations = input("\nWould you like more similar product recommendations? (Y/N)") 
        more_recommendations = more_recommendations.upper().strip() 
    
    if more_recommendations == "Y": 
        print("\nOk. Here are some more product recommendations.") 
        if (len(greatest_similarity_product_ids_list) == 3) | (len(greatest_similarity_product_ids_list) > 3): 
            products_df = df[df["product_id"] == greatest_similarity_product_ids_list[1]] 
        
            print("Product: ", greatest_similarity_product_ids_list[0]) 
            print("Price: ", products_df["price"].iloc[0]) 
            print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 
    
            products_df = df[df["product_id"] == greatest_similarity_product_ids_list[2]] 
            print("Product: ", greatest_similarity_product_ids_list[0]) 
            print("Price: ", products_df["price"].iloc[0]) 
            print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 
    
            products_df = df[df["product_id"] == greatest_similarity_product_ids_list[3]] 
            print("Product: ", greatest_similarity_product_ids_list[0]) 
            print("Price: ", products_df["price"].iloc[0]) 
            print("Average Product Rating: ", products_df["review_score"].mean()) 
    
            good = input("\nWere any of these recommendations helpful? (Y/N)") 
            good = good.upper().strip() 
    
            if good == "Y": 
                another = input("\nWe're glad! Would you like like help finding another product? (Y/N)") 
                another = another.upper().strip() 
                
                if another == "Y": 
                    token_vectors = []; 
                else: 
                    print("\nHave a great day! :)") 
                    break 
            
            else: 
                token_vectors = []; 
                print("\nWe're sorry.") 
    
        elif len(greatest_similarity_product_ids_list) == 2: 
            products_df = df[df["product_id"] == greatest_similarity_product_ids_list[1]] 
        
            print("Product: ", greatest_similarity_product_ids_list[0]) 
            print("Price: ", products_df["price"].iloc[0]) 
            print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 
    
            products_df = df[df["product_id"] == greatest_similarity_product_ids_list[2]] 
            print("Product: ", greatest_similarity_product_ids_list[0]) 
            print("Price: ", products_df["price"].iloc[0]) 
            print("Average Product Rating: ", products_df["review_score"].mean()) 
    
            good = input("\nWere any of these recommendations helpful? (Y/N)") 
            good = good.upper().strip() 
    
            if good == "Y": 
                another = input("\nWe're glad! Would you like like help finding another product? (Y/N)") 
                another = another.upper().strip() 
                
                if another == "Y": 
                    token_vectors = []; 
                else: 
                    print("\nHave a great day! :)") 
                    break 
            
            else: 
                token_vectors = []; 
                print("\nWe're sorry.") 
    
        else: 
            moree = True 
            
            more_detail = input("\nThere are no more similar products. What category is your product in? Enter number.") 
    
            # a dictionary with a number associated with each product category 
            product_categories_dict = {} 
        
            # array with unique product categories 
            product_categories = df["product_cat_name_translated"].unique() 
        
            for i in range(len(product_categories)): 
                product_categories_dict[i+1] = product_categories[i] 
        
            # ask user for product category 
            print("\nWhich category is the product in? Enter in number.") 
            for keys, values in product_categories_dict.items(): 
                print(f"{keys}: {values}") 
            
            user_input_2 = input("") 
            
            user_input_2 = int(user_input_2.strip()) 
            
            user_input_product_cat = product_categories_dict.get(user_input_2) 

    else: 
        if (more_detail_user_input != "Y") & (another != "Y"): 
            print("\nHave a great day! :)") 
            break 
    
    if moree == True: 
        user_product_category = df[df["product_cat_name_translated"] == user_input_product_cat] 
        ranked_products = user_product_category.sort_values(by="review_score", ascending=False) 
        
        print("\nHere are the three highest ranked products in that category.") 
        print(ranked_products.head(3)["product_id"]) 


How about this product?
Product:  10adb53d8faa890ca7c2f0cbcb68d777
Price:  19.9
Average Product Rating:  3.85


In [33]:
print(df[df["product_id"] == "10adb53d8faa890ca7c2f0cbcb68d777"]["payment_installments"]) 
print(df.iloc[17]) 
print(df.iloc[16107]) 

17        3.0
2912      3.0
16107     1.0
17982     3.0
18168     5.0
35359     3.0
36151     3.0
42178     2.0
47354     2.0
49223     1.0
53242     1.0
53563     1.0
64059     1.0
67787     6.0
69422     1.0
81452     1.0
92507     3.0
99816     1.0
100412    1.0
106459    3.0
Name: payment_installments, dtype: float64
order_id                             5ff96c15d0b717ac6ad1f3d77225a350
customer_id                          19402a48fe860416adf93348aba37740
order_status                                                delivered
order_purchase_timestamp                          2018-07-25 17:44:10
order_approved_at                                 2018-07-25 17:55:14
order_delivered_carrier_date                      2018-07-26 13:16:00
order_delivered_customer_date                     2018-07-30 15:52:25
order_estimated_delivery_date                     2018-08-08 00:00:00
review_id                            346e42116863ea64d51cef464d8f0c9c
review_score                                   